In [2]:
import cv2
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms



In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(7 * 7 * 64, 1000),
            nn.Linear(1000, 10)
        )

    def forward(self, x):
        out = self.features(x)
        out = torch.flatten(out, 1) 
        out = self.classifier(out)
        return out



In [4]:
# Load the trained model
model = CNN()
model.load_state_dict(torch.load("C:\\Users\\SWETHA\\conv_net_model.ckpt"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()


CNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=3136, out_features=1000, bias=True)
    (2): Linear(in_features=1000, out_features=10, bias=True)
  )
)

In [5]:
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

True

In [6]:
kernel = np.ones((5, 5), np.uint8)

canvas = None

x1 = 0
y1 = 0

noise_thresh = 800


In [7]:
# Load the HSV values for pen detection
load_from_sys = True

In [8]:
if load_from_sys:
    hsv_value = np.load('hsv_value.npy')
lower_range = hsv_value[0]
upper_range = hsv_value[1]
hsv_value


array([[ 76,  97,   0],
       [179, 255, 255]])

In [9]:
while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)

    if canvas is None:
        canvas = np.zeros_like(frame)

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, lower_range, upper_range)

    mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=2)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours and cv2.contourArea(max(contours, key=cv2.contourArea)) > noise_thresh:
        c = max(contours, key=cv2.contourArea)
        x2, y2, w, h = cv2.boundingRect(c)

        if x1 == 0 and y1 == 0:
            x1, y1 = x2, y2
        else:
            canvas = cv2.line(canvas, (x1, y1), (x2, y2), [0, 255, 255], 4)

        x1, y1 = x2, y2
    else:
        x1, y1 = 0, 0

    # Preprocess the live writing input image
    gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
    resized_gray = cv2.resize(gray, (28, 28))
    resized_gray = resized_gray.astype(np.float32) / 255.0  # Normalize the image
    tensor_img = torch.tensor(resized_gray).unsqueeze(0).unsqueeze(0).to(device)  # Add batch and channel dimensions

    # Use the trained model to predict the digit
    with torch.no_grad():
        model_output = model(tensor_img)
        _, predicted_label = torch.max(model_output, 1)

    digit = predicted_label.item()

    # Display the predicted digit on the screen
    cv2.putText(canvas, f"Predicted Digit: {digit}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Merge the canvas with the frame for display
    frame = cv2.add(frame, canvas)

    stacked = np.hstack((canvas, frame))
    cv2.imshow('Screen_Pen', cv2.resize(stacked, None, fx=0.6, fy=0.6))

    if cv2.waitKey(1) == 13:
        break

    # Clear the canvas when 'c' is pressed
    if cv2.waitKey(1) & 0xFF == ord('c'):
        canvas = None

cv2.destroyAllWindows()
cap.release()



In [10]:
digit # the predicted digit is

8